# Assignment #02 - Regression with BackPropagation


Deep Learning / Fall 1399, Khatam University



---



**Please pay attention to these notes:**
<br><br>



- **Assignment Due:** <b><font color='red'>1399/10/2</font></b> 23:59:00
- If you need any additional information, please review the assignment page on the course website.
- The items you need to answer are highlighted in <font color="purple">**bold purple**</font> and the coding parts you need to implement are denoted by:
```
########################################
#     Put your implementation here     #
########################################
```
- We always recommend co-operation and discussion in groups for assignments. However, **each student has to finish all the questions by him/herself**. If our matching system identifies any sort of copying, you'll be responsible for consequences.
- Students who audit this course should submit their assignments like other students to be qualified for attending the rest of the sessions.
- If you have any questions about this assignment, feel free to drop us a line. You may also post your questions on the course Microsoft Teams channel.
- You must run this notebook on Google Colab platform, it depends on Google Colab VM for some of the depencecies.
- You can double click on collapsed code cells to expand them.
- <b><font color='red'>When you are ready to submit, please follow the instructions at the end of this notebook.</font></b>


<br>





---



# Introduction

In this assignment, you will:
- Get familiar with computational graphs and implement all parts required for creating one.
- Use your implementations to solve a simple regression problem.

In [1]:
#@title Run This Cell
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# 1 . Computational Graph Nodes

To make a computational graph, we need two main types of nodes: 
- **Value** nodes which are either our input values or trainable parameters,
- and **Operation** nodes which are functions that take as input one or more value nodes or the output of other operations.

So, let's implement these two nodes first:

In [3]:
class Value():
  def __init__(self, initial_value=None, trainable=True, graph=None):
    self.value = initial_value
    self.trainable = trainable
    self.upstream_grad = np.array(0)
    if graph:
      graph.values.append(self)
    else:
      _default_graph.values.append(self)
      
  def set_value (self, new_value):
    self.value = new_value

  def get_value (self):
    return self.value

  def set_grad (self, grad):
    self.upstream_grad = grad

As you can see, each value node takes an initial_value as input. Moreover, it has a flag that determines whether this value is trainable (e.g. it is a model parameter) or non-trainable (e.g. it is a model input). We further pass a **graph** object to the value, which is the data structure in which we are going to add this value node. We implement the Graph class later, first let's implement a base class for **Operations** :

In [4]:
class Operation():

  def __init__(self, inputs=None, graph=None):
    self.inputs = inputs
    self.output = None
    self.upstream_grad = np.array(0)
    if graph:
      graph.operations.append(self)
    else:
      _default_graph.operations.append(self)
  
  def get_value (self):
    return self.output
  
  def set_grad (self, grad):
    self.upstream_grad = grad

  def forward (self):
    pass

  def backward (self):
    pass



We can inherit from this base class and implement our desired operations. The `inputs` parameter is a list of inputs (each input can be either a Variable or another Operation) on which our operation is going to function. Implementation of `forward`  and `backward` methods is our main job when we are creating a custom operation. The `forward` method, is the logic of our operation and sets the value of the `output` attribute. The `backward` method is a function that is called during the back-propagation algorithm. To have a better sense, we first explain how the back-propagation algorithm works, and then we implement some custom operations.




# 2 . Forward and Backward Propagation

Let's say we have a simple linear regression problem. We have **(xi, yi)** pairs, and we want to find the optimal weight matrix **W** and the bias term **b** to minimize the **MSE loss** function:

$$ MSE(X, Y) = \frac{1}{N}\sum_1^N {{(||WX + b - Y||}_2)^2}$$

 We can formulate the problem with the following graph:

</br>
</br>
</br>


<p align="center"><img src="https://drive.google.com/uc?id=1WQvu1BwhrinTNBp_6I7aYQt6JEnX7xp6" width="400"/></p>

To minimze the **loss** function, we can use gradient decent algorithm, but first we need to find the drivetive of **loss** with respect to **W** and **b** so we can change them in the direction opposite to the gradients. In order to do so, we can use the chain rule:

- First we can calculate the gradient of **loss** function with respect to its direct inputs, e.g. **ADD** operation. We call this gradient $G_{ADD}(loss)$
- Then we can calculate the gradient of **ADD** with respect to its direct input **b**, we call this gradient $G_{b}(ADD)$
- Finally, we can calculate the gradient of **loss** with respect to **b** using the chain rule:
<p align="center"> $G_{b}(loss) = G_{ADD}(loss) . G_{b}(ADD)$
</br>

By expanding this idea, **back-propagation** calculates the gradient of the loss with respect to all parameters: 

- First, we need to calculate the loss and all its dependency nodes (these values might later going to be used in the calculation of some operation gradients). This step is called the **forward propagation** phase
- Starting from the loss, we first calculate the derivative with respect to its direct inputs
- We then pass these gradients down to these input nodes, and for each node, we calculate the derivative with respect to its direct inputs again, and we pas the upstream gradient multiplied by these new gradients down to the second level nodes
- We continue this process until we reach leaf nodes.

Check [this](http://neuralnetworksanddeeplearning.com/chap2.html) for a detailed explanation of the back-propagation algorithm.


Now let's implement a simple binary-add operation to get a better sense:

In [5]:
class BinaryAdd(Operation):
  
  def __init__(self, inputs=None):
    super().__init__(inputs)
  
  def forward(self):
    a = self.inputs[0].get_value()
    b = self.inputs[1].get_value()
    self.output = a + b

  def backward (self):
    for inp in self.inputs:
      grad_wrt_inp = self.upstream_grad
      while np.ndim(grad_wrt_inp) > len(inp.get_value().shape):
        grad_wrt_inp = np.sum(grad_wrt_inp, axis=0)
      inp.set_grad(grad_wrt_inp)


This operation gets a list with two inputs. As you can observe, the `forward` method simply adds the two inputs and sets the output value of the operation equal to the result, while the `backward` method computes the derivative of operation with respect to its inputs, multiplies these gradients with its received upstream gradient and sets the calculated multiplications as the upstream gradient of its respective input nodes. 

Let's implement another operation, a simple matrix multiplication:

In [6]:
class Matmul(Operation):
  
  def __init__(self, inputs=None):
    super().__init__(inputs)
  
  def forward(self):
    a = self.inputs[0].get_value()
    b = self.inputs[1].get_value()
    self.output = a.dot(b)

  def backward (self):
    a = self.inputs[0].get_value()
    b = self.inputs[1].get_value()
    self.inputs[0].set_grad(self.upstream_grad.dot(b.T))
    self.inputs[1].set_grad(a.T.dot(self.upstream_grad))

This operation takes as input a list of 2D-matrices (operations or values with output rank of 2) and calculates the matrix multiplication of them. The `forward` method is pretty straight-forward, but let's get into details of the `backward` method:

- Let's say we feed a matrix **$X$** with shape **$m\times{k}$** and a matrix **$W$** with shape **$k\times{n}$** to the operation. The output **$O$** will be a matrix with the shape of **$m\times{n}$**.

- The upstream gradient **$G$** has the same shape as **$O$** (this holds for the upstream gradient of all the nodes since the upstream gradient is the gradient of the loss with respect to node outputs).

- Let's say we want to calculate the gradient of loss with respect to  **$X$**: The upstream gradient with respect to the **$(i,j)$**-th element of **$X$** will be **$\sum_{l=1}^n{G_{i,l}\times{W_{j,l}}}$** (You can write this down to see it yourself). Therefore we can simply write the gradient with respect to **$X$** as **$G{W^{T}}$**. 

- Similarly, we can write the gradient with respect to **$W$** as **${X^{T}}G$**. 

<b><font color='purple'>Now it is your turn to implement the MSE operation</font></b>, which takse as input a list of 2D-matrices with the same shape, and calculates the following value as the output:

</br>
$$ MSE(X, Y) = \frac{1}{N}\sum_{all} {(X - Y)^2}$$

In which **N** is the first dimension of **X** and **Y** (which is usually equal to batch size, if we consider our input shapes to be  $batch\_size\times{sample\_size}$)

- Note that if we want to use MSE as our loss, we can set the MSE upstream gradient to be equal to 1 since the gradient of the loss with respect to itself will be 1, but you should treat MSE as a regular operation in your implementation and take its upstream gradient into consideration regardless of whether we set it to 1 or not. 


In [7]:
#@title Your Part #1
class MSE(Operation):

  def __init__(self, inputs=None):
    super().__init__(inputs)
  
  def forward(self):
    ########################################
    #     Put your implementation here     #
    ########################################

  def backward (self):
    ########################################
    #     Put your implementation here     #
    ########################################

#3 . Graph Iteration 

Now we implement the **Graph** class that we saw before. `Graph` is the data structure in which we append our operations and values. In order to add an operation or value into a `Graph` instance, we have to pass it when instantiating an `Operation` or a `Value` (refer to the implementation of them). If we call the `set_as_default` method of a `Graph` instance, nodes will be automatically added to that instance by default (refer to the implementation of `Operation` and `Value` again).

 `Graph` has two important methods: `forward_topology_ordering` and `backward_topology_ordering`. In order to evaluate the output of a node, we need to call its `forward` method, however the `forward` method of its inputs must have been called before this, and this goes recursively down until the inputs of a node are value nodes which their output is already prepared. Therefore, we need a list of nodes required to be called in order, starting from value nodes, before we can call the `forward` method of our target node. The `forward_topology_ordering` method finds this ordering for any given node. 
 
This procedure is reversed for the `backward` method: To call the `backward` method of a node, the `backward` method of its outputs must have been called, and so on. Hence `backward_topology_ordering`, finds the ordering of nodes in which we must call the `backward` methods, starting from a node back to the value nodes. Note that value nodes are not present in either ordering, since they have neither a `forward` method nor a `backward` method to be called. 

Here we implement `forward_topology_ordering` using a simple DFS algorithm. <b><font color='purple'> You must implement `backward_topology_ordering` using the BFS algorithm. </font></b> 

In [8]:
#@title Your Part # 2
class Graph():
  def __init__(self):
    self.operations = []
    self.values = []

  def set_as_default(self):
    global _default_graph
    _default_graph = self

  def forward_topology_ordering(self, operation):
    ordering = []
    visited_nodes = set()

    def recursive_helper(node):
      if isinstance(node, Operation):
        for input_node in node.inputs:
          if input_node not in visited_nodes:
            recursive_helper(input_node)
        ordering.append(node)
      visited_nodes.add(node)

    recursive_helper(operation)
    return ordering

  def forward(self, operation):
    ordering = self.forward_topology_ordering(operation)
    for node in ordering:
      node.forward()
    return operation.get_value()

  def backward_topology_ordering(self, operation):
    
    ########################################
    #     Put your implementation here     #
    ########################################


  def backward(self, loss_operation):

    # The upstream gradient of loss function is always 1 since 
    # the derivative of loss with respect to its output is 1
    loss_operation.set_grad (np.array(1))
    ordering = self.backward_topology_ordering(loss_operation)
    for node in ordering:
      node.backward()
    

# 4 . Optimizer

One last piece is remaining before we can complete our puzzle, which is the **Optimizer**. The `Optimizer` has the responsibility of changing the values of our model parameters with respect to their gradients. Each optimizer may have its own update rule, for example, a simple Gradient Decent optimizer simply changes the values in the opposite direction of the gradient with a step size which we call the learning rate. Here we implement a gradient descent optimizer, pay attention to the implementation details since you are going to implement another optimizer a bit later. 

In [9]:
class Optimizer():
  def __init__ (self):
    self.history = []

  def reset_history(self):
    self.history = []


class GD(Optimizer):
  def __init__ (self, learning_rate):
    super().__init__()
    self.learning_rate = learning_rate
    
  def optimize (self, graph, loss_operation):
    ## Forward Propagation
    l = graph.forward (loss_operation)
    self.history.append(l)
    ## Backward Propagation
    graph.backward (loss_operation)
    ## Variable Optimization
    for v in graph.values:
      if v.trainable:
        v.set_value (v.get_value()-self.learning_rate*v.upstream_grad)

# 5 . Linear Regression

Nice, we now have all the pieces! Let's solve a simple linear regression problem to check if our implementations work.

First, we create artificial data:

In [ ]:
X = np.linspace(-10.0, 10.0, num=100)
Y =  (2*(X-3)*(X+4)*(X) - X)+(np.random.rand((100))*500)-250



plt.plot(X, Y, ".")
plt.show()

Now let's find the best line we can draw to fit these data points:

In [ ]:
G = Graph()
G.set_as_default()

initial_w = 40.0
initial_b = -1500.0

w = Value(initial_value=np.array([[initial_w]]))
b = Value(initial_value=np.array([initial_b]))

inp_x = Value(trainable=False)
inp_y = Value(trainable=False)

mul = Matmul([inp_x, w])
add = BinaryAdd([mul, b])

loss = MSE([add, inp_y])

gd_opt = GD(0.0001)
gd_opt.reset_history()

for itter in tqdm(range (20000)):
  inp_x.set_value (X.reshape((-1,1)))
  inp_y.set_value (Y.reshape((-1,1)))

  gd_opt.optimize (G, loss)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(10,5))

X_0_0 , Y_0_0 = -10, -10*initial_w + initial_b
X_100_0 , Y_100_0 = 10, 10*initial_w + initial_b
ax[0].plot(X, Y, ".")
ax[0].plot([X_0_0, X_100_0], [Y_0_0, Y_100_0])
ax[0].axis('off')
ax[0].set_title("Before")


X_0_1 , Y_0_1 = -10, -10*w.value[0][0] + b.value[0]
X_100_1 , Y_100_1 = 10, 10*w.value[0][0] + b.value[0]
ax[1].plot(X, Y, ".")
ax[1].plot([X_0_1, X_100_1], [Y_0_1, Y_100_1])
ax[1].axis('off')
ax[1].set_title("After")

plt.show()

# 6 . Gradient Descent with Momentum

There is an extension to the gradient descent called gradient descent with momentum. So instead of updating the parameters based only on current gradients, we also take into account the gradients from previous steps! 

$$ w \leftarrow w - \alpha (\nabla_wLoss(i) + \gamma  \nabla_wLoss(i-1)) $$

In which $\alpha$ is our learning rate, $\gamma$ is the momentum and $\nabla_wLoss(i)$ denotes the gradient of $Loss$ with respect to $w$ at step $i$.

<b><font color='purple'>Now it is your turn to implement gradient descent with momentum optimizer!</font></b>

In [13]:
#@title Your Part #3
class MomentumGD(Optimizer):
  
  def __init__ (self, learning_rate, momentum):
    self.learning_rate = learning_rate
    self.momentum = momentum

  def optimize (self, graph, loss_operation):
    ## Forward Propagation
    l = graph.forward (loss_operation)
    self.history.append(l)
    
    ########################################
    #     Put your implementation here     #
    ########################################

Now let's solve our regression problem with this new optimizer once again:

In [ ]:
G = Graph()
G.set_as_default()

initial_w = 40.0
initial_b = -1500.0

w = Value(initial_value=np.array([[initial_w]]))
b = Value(initial_value=np.array([initial_b]))

inp_x = Value(trainable=False)
inp_y = Value(trainable=False)

mul = Matmul([inp_x, w])
add = BinaryAdd([mul, b])

loss = MSE([add, inp_y])

mgd_opt = MomentumGD(0.0001, 0.7)
mgd_opt.reset_history()

for itter in tqdm(range (20000)):
  inp_x.set_value (X.reshape((-1,1)))
  inp_y.set_value (Y.reshape((-1,1)))

  mgd_opt.optimize (G, loss)

Let's check how these optimizers perform compared to each other, by visualizing the loss history:

In [ ]:
plt.plot(range(20000), gd_opt.history[:], label="Regular")
plt.plot(range(20000), mgd_opt.history[:], label="With Momentum")

plt.legend(loc="upper right")

plt.xlabel('Itteration')
plt.ylabel('Loss')

plt.show()

As you can observe, momentum gradient descent converges much faster.


<b><font color='purple'>What is the reason?</font></b>


<b><font color='purple'> -- PUT YOUR ANSWER HERE! -- </font></b>

# Submission

Congratulations! You finished the assignment & you're ready to submit your work. Please follow the instructions:

1. Check and review your answers. Make sure all of the cell outputs are what you want. 
2. Select File > Save.
3. **Fill your information** & run the cell bellow.
4. Run **Make Submission** cell, It may take several minutes and it may ask you for your credential.
5. Run **Download Submission** cell to obtain your submission as a zip file.
6. Grab the downloaded file (`dl_asg02__xx__xx.zip`) and hand it over in microsoft teams.

## Fill your information (Run the cell)

In [ ]:
#@title Enter your information & "RUN the cell!!" { run: "auto" }
student_id = "" #@param {type:"string"}
student_name = "" #@param {type:"string"}

print("your student id:", student_id)
print("your name:", student_name)


from pathlib import Path

ASSIGNMENT_PATH = Path('asg02')
ASSIGNMENT_PATH.mkdir(parents=True, exist_ok=True)

## Make Submission (Run the cell)

In [ ]:
#@title Make submission
! pip install -U --quiet PyDrive > /dev/null
! pip install -U --quiet jdatetime > /dev/null

# ! wget -q https://github.com/github/hub/releases/download/v2.10.0/hub-linux-amd64-2.10.0.tgz 


import os
import time
import yaml
import json
import jdatetime

from google.colab import files
from IPython.display import Javascript
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

asg_name = 'Assignment_2'
script_save = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
# repo_name = 'iust-deep-learning-assignments'
submission_file_name = 'dl_asg02__%s__%s.zip'%(student_id, student_name.lower().replace(' ',  '_'))

sub_info = {
    'student_id': student_id,
    'student_name': student_name, 
    'dateime': str(jdatetime.date.today()),
    'asg_name': asg_name
}
json.dump(sub_info, open('info.json', 'w'))

Javascript(script_save)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = drive.ListFile({'q':"title='%s.ipynb'"%asg_name}).GetList()[0]['id']
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('%s.ipynb'%asg_name) 

! jupyter nbconvert --to script "$asg_name".ipynb > /dev/null
! jupyter nbconvert --to html "$asg_name".ipynb > /dev/null
! zip "$submission_file_name" "$asg_name".ipynb "$asg_name".html "$asg_name".txt info.json > /dev/null

print("##########################################")
print("Done! Submisson created, Please download using the bellow cell!")

In [ ]:
files.download(submission_file_name)